In [ ]:
import numpy as np
import numba
import matplotlib.pyplot as plt
import pandas as pd 
import os 
from tqdm import tqdm 

In [ ]:
data_path = os.path.join(os.getcwd(), os.pardir, "data", "emerson", "emerson_processed")
whole_data = pd.read_csv(os.path.join(data_path, "whole_seqs_nn.tsv"), sep = '\t')

In [ ]:
train_data = pd.read_csv(os.path.join(data_path, "whole_seqs_nn_train.tsv"), sep = '\t')
#train_data = whole_data

In [ ]:
len(train_data)

### V- and J-gene distribution

In [ ]:
v_genes = train_data.loc[:,"v"]
j_genes = train_data.loc[:,"j"]

In [ ]:
@numba.jit(forceobj=True)
def create_combinations(data):
    result = []
    for i in tqdm(range(len(data)),position=0, leave=True):
        result.append(f'{data[i, 3]}, {data[i, 6]}')
    return result 

In [ ]:
v_j_combinations = create_combinations(train_data.to_numpy())

In [ ]:
gene_count = {}
for comb in tqdm(v_j_combinations, total = train_data.shape[0], position=0, leave=True):
    key = str(comb)
    if key not in gene_count.keys():
        gene_count[key] = 1
    else:
        gene_count[key] += 1
gene_count.update((x, y/train_data.shape[0]) for x, y in gene_count.items())

In [ ]:
gene_count_sorted = dict(sorted(gene_count.items(), key = lambda x: x[1], reverse= True))

In [ ]:
sum(gene_count_sorted.values())


In [ ]:
fig = plt.figure(figsize=(50,50))
plt.bar(x=list(gene_count_sorted.keys()), height= list(gene_count_sorted.values()), align='edge')
plt.xticks(rotation=-90)
plt.show

In [ ]:
indx = int(len(gene_count_sorted)/7)

In [ ]:
def gene_combination(begin_indx, end_indx):
    print(pd.DataFrame(data={'gene combination': list(gene_count_sorted.keys())[begin_indx:end_indx], 'fraction (%)': [f'{i*100:.3}'for i in list(gene_count_sorted.values())[begin_indx:end_indx]]}).to_latex(index=False))

In [ ]:
gene_combination(0, indx)
gene_combination(indx, 2*indx)
gene_combination(2*indx, 3*indx)
gene_combination(3*indx, 4*indx)
gene_combination(4*indx, 5*indx)
gene_combination(5*indx, 6*indx)
gene_combination(6*indx, len(gene_count_sorted))

### distribution of deletions of v gene  

In [ ]:
v_deletions= train_data.loc[:,"v_deletions"]
j_deletions = train_data.loc[:,"j_deletions"]

In [ ]:
v_deletion_count = {}
for comb in tqdm(v_deletions, total = train_data.shape[0], position=0, leave=True):
    key = str(comb)
    if key not in v_deletion_count.keys():
        v_deletion_count[key] = 1
    else:
        v_deletion_count[key] += 1
v_deletion_count.update((x, y/train_data.shape[0]) for x, y in v_deletion_count.items())

In [ ]:
v_deletion_count_sorted = dict(sorted(v_deletion_count.items(), key= lambda x: x[1], reverse=True))
v_deletion_count_sorted

In [ ]:
sum(list(v_deletion_count_sorted.values()))

In [ ]:
print(pd.DataFrame(data={'number of deletions': list(v_deletion_count_sorted.keys()), 'fraction (%)': [f'{i *100:.3}' for i in v_deletion_count_sorted.values()]}).to_latex(index=False))

In [ ]:
sum(list(v_deletion_count_sorted.values())[:11])

In [ ]:
list(v_deletion_count_sorted.keys())[:11]

In [ ]:
v_deletion_count_sorted = {int(k):v for k,v in v_deletion_count_sorted.items()}
v_deletion_count_sorted = dict(sorted(v_deletion_count_sorted.items(), reverse=False))

In [ ]:
fig = plt.figure(figsize = (10,5))
plt.bar(v_deletion_count_sorted.keys(), v_deletion_count_sorted.values())
plt.title("Deletions on V gene")
plt.xlabel("No. of deletions")
plt.ylabel("Fraction of the train set")
plt.show
plt.savefig(os.path.join("figures", "V_deletion_train.png"))

### distribution of deletions on J - genes  

In [ ]:
j_deletion_count = {}
for comb in tqdm(j_deletions, total = train_data.shape[0], position=0, leave=True):
    key = str(comb)
    if key not in j_deletion_count.keys():
        j_deletion_count[key] = 1
    else:
        j_deletion_count[key] += 1
j_deletion_count.update((x, y/train_data.shape[0]) for x, y in j_deletion_count.items())

In [ ]:
j_deletion_count_sorted = dict(sorted(j_deletion_count.items(), key= lambda x: x[1], reverse=True))
j_deletion_count_sorted

In [ ]:
print(pd.DataFrame(data={'number of deletions': list(j_deletion_count_sorted.keys()), 'fraction (%)': [f'{i *100:.3}' for i in j_deletion_count_sorted.values()]}).to_latex(index=False))

In [ ]:
sum(list(j_deletion_count_sorted.values())[:12])

In [ ]:
list(j_deletion_count_sorted.keys())[:12]

In [ ]:
j_deletion_count_sorted = {int(k):v for k,v in j_deletion_count_sorted.items()}
j_deletion_count_sorted = dict(sorted(j_deletion_count_sorted.items(), reverse=False))

In [ ]:
fig = plt.figure(figsize = (10,5))
plt.bar(j_deletion_count_sorted.keys(), j_deletion_count_sorted.values())
plt.title("Deletions on J gene")
plt.xlabel("No. of deletions")
plt.ylabel("Fraction of the train set")
plt.show
plt.savefig(os.path.join("figures", "J_deletion_train.png"))

### Distribution of Lengths of cdr3 sequences 

In [ ]:
cdr3 = train_data.loc[:, 'seq']

In [ ]:
@numba.jit(forceobj=True)
def count_seq_len(data):
    result = []
    for i in tqdm(range(len(data)),position=0, leave=True):
        result.append(len(data[i]))
    return result

In [ ]:
cdr3_lengths = count_seq_len(cdr3.to_numpy())

In [ ]:
cdr3_len_count = {}
for len_ in tqdm(cdr3_lengths, total = train_data.shape[0], position=0, leave=True):
    key = str(len_)
    if key not in cdr3_len_count.keys():
        cdr3_len_count[key] = 1
    else:
        cdr3_len_count[key] += 1
cdr3_len_count.update((x, y/train_data.shape[0]) for x, y in cdr3_len_count.items())

In [ ]:
cdr3_len_count

In [ ]:
sum(list(cdr3_len_count.values()))

In [ ]:
sorted_cdr3_len = dict(sorted(cdr3_len_count.items(), key = lambda x: x[1], reverse = True))

In [ ]:
sum(list(sorted_cdr3_len.values())[:9])

In [ ]:
list(sorted_cdr3_len.keys())[:9]

In [ ]:
print(pd.DataFrame(data={'len of cdr3': sorted_cdr3_len.keys(), 'fraction (%)': [f'{i*100:.3}'for i in sorted_cdr3_len.values()]}).to_latex(index=False))

In [ ]:
sorted_cdr3_len = {int(k):v for k,v in sorted_cdr3_len.items()}
sorted_cdr3_len= dict(sorted(sorted_cdr3_len.items(), reverse=False))

In [ ]:
fig = plt.figure(figsize = (10,5))
plt.bar(sorted_cdr3_len.keys(), sorted_cdr3_len.values())
plt.title("Lengths of CDR3 sequences")
plt.xlabel("Length of CDR3 seq")
plt.ylabel("Fraction of the train set")
plt.show
plt.savefig(os.path.join("figures","CDR3_train.png"))

### Specific V gene deletions 

In [ ]:
v_genes_deletions = train_data.iloc[:,['v', 'v_deletions']]